<a href="https://colab.research.google.com/github/bradsprigg/prompt-explorer/blob/main/CLIP_Prompt_Synonymiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This group installs and loads things
Open this up if you want to change CLIP's model or some other guts, otherwise just run it.

In [1]:
%%capture
%pip install --no-deps git+https://github.com/openai/CLIP.git
%pip install --no-deps ftfy regex tqdm

In [2]:
import torch
import gc
import warnings
warnings.filterwarnings('ignore')
torch.set_grad_enabled(False)

def clear_mem():
    torch.cuda.empty_cache()
    gc.collect()

In [3]:
import clip
torch.set_grad_enabled(False)

perceptor, clip_preprocess = clip.load('ViT-B/16')
#perceptor, clip_preprocess = clip.load('ViT-B/32')
perceptor.eval().float().requires_grad_(False);

tokenizer = clip.simple_tokenizer.SimpleTokenizer()

100%|████████████████████████████████████████| 335M/335M [00:02<00:00, 136MiB/s]


In [4]:
def similar_words(target_word="cool", topk=32):
    target_tokens = tokenizer.encode(target_word)
    if len(target_tokens) > 1:
        print("This word uses more than one token, can't use it!")
        return
    target_emb = perceptor.token_embedding.weight[target_tokens[0],None].detach()
    token_sim  = torch.cosine_similarity(target_emb,perceptor.token_embedding.weight.detach(),-1)
    top_token_sim = torch.topk(token_sim,topk+1,-1,True,True)
    top_indices = top_token_sim.indices[1:]
    top_values  = top_token_sim.values[1:]
    for i in range(top_indices.shape[0]):
        print('"'+tokenizer.decode([top_indices[i].item()])+'"',"   ", top_values[i].item())
    return

In [5]:
from IPython.display import HTML, display
from random import sample

def similar_prompt_fancy(target_word="cool", topk=32):
    target_tokens = tokenizer.encode(target_word)
    new_prompt = ""
    for now_token in target_tokens:
        target_emb = perceptor.token_embedding.weight[now_token,None].detach()
        token_sim  = torch.cosine_similarity(target_emb,perceptor.token_embedding.weight.detach(),-1)
        top_token_sim = torch.topk(token_sim,topk+1,-1,True,True)
        top_indices = top_token_sim.indices[1:]
        top_values  = top_token_sim.values[1:]
        output = []
        for i in range(top_indices.shape[0]):
            output.append([tokenizer.decode([top_indices[i].item()]), top_values[i].item()]) 
        new_token = sample(output, len(output))[0][0]
        new_prompt += new_token
    print(new_prompt)

### Play Area

In [9]:
#@title Prompt Synonyms
#@markdown Breaks the prompt down into tokens and then finds similar tokens for each one, selecting one at random 
#@markdown within topk of each. Concatenates them into a new prompt. Just wanted to see what would happen really.
#@markdown Repeats how_many times.

prompt = "a group of ravers on ecstasy at an underground dance party" #@param {type:"string"}
topk = 8 #@param {type:"integer"}
how_many = 5 #@param {type:"integer"}

#similar_words(word, topk)
for x in range(0, how_many):
    similar_prompt_fancy(prompt, topk)

one grouwith raistevens for perfuon some ground dances party
his grouin raiwer onto alloy in the geothermal dances aparty 
my team , tawer to turbulof a resistance dancingparties 
one meeting by tares onalloy for some ground dancparties 
one meeting with rottes of victory of some geothermal dancers dinner 
